In [67]:
import random
import logging
import sys
import numpy as np
from typing import Callable
import heapq
from collections import Counter
logging.basicConfig(format="%(message)s", level=logging.INFO)

In [68]:
N=20
MIN_NUMBER = sys.float_info.min
#print(MIN_NUMBER)

In [69]:
# Copyright © 2022 Giovanni Squillero <squillero@polito.it>
# https://github.com/squillero/computational-intelligence
# Free for personal or classroom use; see 'LICENSE.md' for details.


class PriorityQueue:
    """A basic Priority Queue with simple performance optimizations"""

    def __init__(self):
        self._data_heap = list()
        self._data_set = set()

    def __bool__(self):
        return bool(self._data_set)

    def __contains__(self, item):
        return item in self._data_set

    def push(self, item, p=None):
        assert item not in self, f"Duplicated element"
        if p is None:
            p = len(self._data_set)
        self._data_set.add(item)
        heapq.heappush(self._data_heap, (p, item))

    def pop(self):
        p, item = heapq.heappop(self._data_heap)
        self._data_set.remove(item)
        return item

In [70]:
def problem(N, seed=42):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [71]:
GOAL={i for i in range(N)}
print(GOAL)
list_of_lists=problem(N)
print(list_of_lists)
N_of_lists=len(list_of_lists)
print(len(list_of_lists))


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
[[8, 4, 7], [0, 1, 2, 3, 6, 13, 17, 18], [0, 6, 16, 17, 19], [0, 5, 7, 8, 13, 14, 17, 18], [2, 3, 4, 6, 8, 10], [1, 3, 8, 11, 14, 19], [2, 3, 9, 11, 12, 17, 18, 19], [1, 2, 9, 7], [3, 5, 6, 7, 8, 11, 12, 14], [2, 5, 7, 8, 12, 14, 17, 19], [17, 10, 1, 7], [2, 6, 8, 10, 12, 15, 18], [4, 7, 8, 14, 17, 18], [4, 7, 11, 12, 15, 16, 18], [1, 3, 4, 5], [2, 8, 12, 13, 14, 16, 17, 19], [0, 3, 5, 8, 9, 10, 13, 14, 17], [8, 16, 5], [16, 9, 19, 6], [0, 5, 11, 16, 17], [0, 1, 3, 7, 9, 10, 11, 15], [18, 2, 15], [4, 5, 8, 13, 15, 16, 17, 19], [6, 9, 11, 12, 17], [2, 3, 7, 10, 14, 16], [17, 18, 7], [0, 1, 2, 7], [16, 10, 2, 7], [4, 6, 15, 17, 18], [3, 6, 7, 13, 15], [1, 3, 13, 14], [3, 6, 7, 10, 14, 17], [5, 7, 8, 13, 14], [0, 1, 2, 3, 5, 7, 14, 17]]
34


In [72]:
def concat_list_of_lists(state):
    c_list = []
    for n in state:
        c_list += list_of_lists[n]
    return c_list

def print_lists(state):
    for n in state:
        print("List number: ",n, " with elements ", list_of_lists[n])
        

In [73]:
def goal_test(state):
    if set(concat_list_of_lists(state)) == GOAL:
        return True
    return False

In [74]:
def result(state):
    new_states=[]
    for n in range(state[len(state)-1]+1, len(list_of_lists)):
        N=(n,)
        new_state=state+N
        new_states.append(new_state)
    return new_states

#state=(23,)
#print(action(state))

In [75]:
def calculate_cost(state):
    c_list=concat_list_of_lists(state)
    repetitions=len(c_list)-len(set(c_list))
    if repetitions==0:
        repetitions=MIN_NUMBER
    normalized_cost=repetitions/len(c_list)
    return normalized_cost
    

In [76]:
def search(
    goal_test: Callable,
    state_cost: dict,
    priority_function: Callable,
    calculate_cost: Callable,
):  
    #initialize collections
    frontier = PriorityQueue()
    state_cost.clear()
    #keeping track of the iterations/nodes
    n=1
    N_nodes=len(list_of_lists)
    #INITIAL_STATE
    state = (0,)
    state_cost[hash(state)] = calculate_cost(state)
    for s in range(1,len(list_of_lists)):
        S=(s,)
        state_cost[hash(S)] = calculate_cost(S)
        #print(S)
        #print(state_cost[hash(S)])
        frontier.push(S,p=priority_function(S))
    #print(state)
    #print(state_cost[hash(state)])
        
    #BEGIN OF THE LOOP
    while state is not None and not goal_test(state):
        n+=1
        new_states = result(state)
        for new_state in new_states:
            if hash(new_state) not in state_cost and new_state not in frontier:
                state_cost[hash(new_state)] = calculate_cost(new_state) 
                frontier.push(new_state, p=priority_function(new_state))
                N_nodes+=1
                #logging.debug(f"Added new node to frontier (cost={state_cost[hash(new_state)]})")
        if frontier:
            state = frontier.pop()
        else:
            state = None
    
    #OUTPUT       
    if state is not None:
        c_list=concat_list_of_lists(state)
        repetitions=len(c_list)-len(set(c_list))
        logging.info(f"Found a solution for N={N:,} with {n:,} processed states, with {N_nodes:,} nodes added in the frontier and with {repetitions:,} repetitions.")
        print("The lists are:")
        print_lists(state)
    else:
        logging.info(f"No solution found")
    

In [77]:
"""
#frontier=[[0,6],[0,6,9],[2,3],[9,8],[5,1],[5]]
#state=[0]
#frontier=[[1],[2],[3],[4]]
#state=[1]
#frontier=[[2],[3],[4],[0, 1], [0, 2], [0, 3], [0, 4]]
#state=[2]
#frontier=[[3],[4],[0, 1], [0, 2], [0, 3], [0, 4],[1, 2], [1, 3], [1, 4]]
#state=[3]
#frontier=[[4],[0, 1], [0, 2], [0, 3], [0, 4],[1, 2], [1, 3], [1, 4],[2, 3], [2, 4]]
#state=[4]
#frontier=[[0, 1], [0, 2], [0, 3], [0, 4],[1, 2], [1, 3], [1, 4],[2, 3], [2, 4],[3, 4]]
#state=[0,1]
#frontier=[[0, 2], [0, 3], [0, 4],[1, 2], [1, 3], [1, 4],[2, 3], [2, 4],[3, 4]]

#new_states=action(state)
#print(new_states)
"""
"""
state=[8,9,11]
print(goal_test(state))
print(calculate_cost(state))
"""
state_cost = dict()

search(
    goal_test=goal_test,
    state_cost=state_cost,
    priority_function=lambda s: state_cost[hash(s)],
    calculate_cost=calculate_cost,
)


Found a solution for N=20 with 1,350 processed states, with 10,584 nodes added in the frontier and with 3 repetitions.


The lists are:
List number:  0  with elements  [8, 4, 7]
List number:  11  with elements  [2, 6, 8, 10, 12, 15, 18]
List number:  18  with elements  [16, 9, 19, 6]
List number:  19  with elements  [0, 5, 11, 16, 17]
List number:  30  with elements  [1, 3, 13, 14]
